In [114]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm

In [115]:
class PlayingCardDataset(Dataset):
  def __init__(self, data_dir, transform=None):
    self.data= ImageFolder(data_dir, transform=transform)

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]

  @property
  def classes(self):
    return self.data.classes

In [116]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gpiosenka/cards-image-datasetclassification")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/cards-image-datasetclassification


In [117]:
dataset = PlayingCardDataset(data_dir='/kaggle/input/cards-image-datasetclassification/train')

In [118]:
len(dataset)

7624

In [119]:
target_to_class = {v: k for k, v in ImageFolder('/kaggle/input/cards-image-datasetclassification/train').class_to_idx.items()}

In [120]:
target_to_class

{0: 'ace of clubs',
 1: 'ace of diamonds',
 2: 'ace of hearts',
 3: 'ace of spades',
 4: 'eight of clubs',
 5: 'eight of diamonds',
 6: 'eight of hearts',
 7: 'eight of spades',
 8: 'five of clubs',
 9: 'five of diamonds',
 10: 'five of hearts',
 11: 'five of spades',
 12: 'four of clubs',
 13: 'four of diamonds',
 14: 'four of hearts',
 15: 'four of spades',
 16: 'jack of clubs',
 17: 'jack of diamonds',
 18: 'jack of hearts',
 19: 'jack of spades',
 20: 'joker',
 21: 'king of clubs',
 22: 'king of diamonds',
 23: 'king of hearts',
 24: 'king of spades',
 25: 'nine of clubs',
 26: 'nine of diamonds',
 27: 'nine of hearts',
 28: 'nine of spades',
 29: 'queen of clubs',
 30: 'queen of diamonds',
 31: 'queen of hearts',
 32: 'queen of spades',
 33: 'seven of clubs',
 34: 'seven of diamonds',
 35: 'seven of hearts',
 36: 'seven of spades',
 37: 'six of clubs',
 38: 'six of diamonds',
 39: 'six of hearts',
 40: 'six of spades',
 41: 'ten of clubs',
 42: 'ten of diamonds',
 43: 'ten of hear

In [121]:
# Transforming the dataset
transform= transforms.Compose([transforms.Resize((128,128)), transforms.ToTensor(),])

In [122]:
data_dir = '/kaggle/input/cards-image-datasetclassification/train'
dataset = PlayingCardDataset(data_dir, transform)  #Uploading the transformed dataset

In [123]:
# Printing a random card image and the corresponding class that it belongs to
image, label = dataset[100]
image


tensor([[[0.0000, 0.0039, 0.0078,  ..., 0.0039, 0.0039, 0.0039],
         [0.0039, 0.0078, 0.0431,  ..., 0.0196, 0.0039, 0.0078],
         [0.0235, 0.0392, 0.3490,  ..., 0.2392, 0.0314, 0.0118],
         ...,
         [0.0039, 0.0235, 0.2392,  ..., 0.2196, 0.0314, 0.0118],
         [0.0039, 0.0039, 0.0235,  ..., 0.0118, 0.0078, 0.0039],
         [0.0039, 0.0078, 0.0039,  ..., 0.0039, 0.0039, 0.0039]],

        [[0.0078, 0.0078, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
         [0.0039, 0.0039, 0.0392,  ..., 0.0196, 0.0039, 0.0078],
         [0.0078, 0.0275, 0.3373,  ..., 0.2392, 0.0275, 0.0078],
         ...,
         [0.0039, 0.0235, 0.2431,  ..., 0.2196, 0.0314, 0.0078],
         [0.0039, 0.0039, 0.0235,  ..., 0.0157, 0.0078, 0.0039],
         [0.0039, 0.0078, 0.0039,  ..., 0.0078, 0.0078, 0.0039]],

        [[0.0196, 0.0196, 0.0118,  ..., 0.0118, 0.0118, 0.0118],
         [0.0118, 0.0078, 0.0353,  ..., 0.0157, 0.0039, 0.0078],
         [0.0118, 0.0196, 0.3137,  ..., 0.2196, 0.0235, 0.

In [124]:
image.shape  #Converted to vector

torch.Size([3, 128, 128])

In [125]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [126]:
# To retreive the first batch of data and then loop over it
for images, labels in dataloader:
    break

In [127]:
images.shape  #32 inputs in one batch

torch.Size([32, 3, 128, 128])

In [128]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2= nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2= nn.MaxPool2d(kernel_size=2, stride=2)


        self.fc1= nn.Linear(32*32*32, num_classes)
        self.dropout = nn.Dropout(0.5)



    def forward(self, x):
        x=self.pool1(self.relu1(self.conv1(x)))
        x=self.pool2(self.relu2(self.conv2(x)))
        x=x.view(x.size(0), -1)  # Flattening 3D features into 1D vector
        x=self.fc1(x)
        x= self.dropout(x)
        return x

In [129]:
num_classes= len(dataset.classes)
print(num_classes)

model= CNN(num_classes= num_classes)

53


In [130]:
cross_entropy_loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [131]:
model.to("cpu")

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=32768, out_features=53, bias=True)
)

In [132]:
num_epochs = 5
model.train()

for epoch in range(num_epochs):
    running_loss = 0.0

    for images, labels in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images = images.to("cpu")
        labels = labels.to("cpu")

        optimizer.zero_grad()

        outputs = model(images)
        loss = cross_entropy_loss(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(dataloader)
    print(f"✅ Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")


Epoch 1/5:   0%|          | 0/239 [00:00<?, ?it/s]

✅ Epoch 1/5, Average Loss: 2.7467


Epoch 2/5:   0%|          | 0/239 [00:00<?, ?it/s]

✅ Epoch 2/5, Average Loss: 1.6365


Epoch 3/5:   0%|          | 0/239 [00:00<?, ?it/s]

✅ Epoch 3/5, Average Loss: 0.9347


Epoch 4/5:   0%|          | 0/239 [00:00<?, ?it/s]

✅ Epoch 4/5, Average Loss: 0.4507


Epoch 5/5:   0%|          | 0/239 [00:00<?, ?it/s]

✅ Epoch 5/5, Average Loss: 0.2145


In [133]:
model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for images, labels in dataloader:
        images = images.to("cpu")
        labels = labels.to("cpu")

        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate accuracy
accuracy = 100 * correct / total
print(f"Accuracy on training data: {accuracy:.2f}%")


Accuracy on training data: 97.39%


In [135]:
# Load test set
test_data_dir = '/kaggle/input/cards-image-datasetclassification/test'
test_dataset = PlayingCardDataset(test_data_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32)


In [137]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = correct / total
print(f" Test Accuracy: {test_accuracy:.4f}")


 Test Accuracy: 0.6981
